# Rolling Ratings

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/boxscores_team/"

In [ ]:
def rolling_val(df, val):
    df["r" + val] = (
        df.apply(lambda x: x[val] * x["poss"], axis=1).cumsum() / df["tposs"]
    )
    return df

In [ ]:
def get_rolling_ratings(season=2023, remove_games_start=0, remove_games_end=0):
    df1 = pd.read_csv(box_DIR + f"NBA_BoxScores_Adv_{season}.csv")
    df1 = df1.rename(
        columns={
            "offensiveRating": "ORtg",
            "defensiveRating": "DRtg",
            "netRating": "NRtg",
            "possessions": "poss",
        }
    )
    cols = ["gameId", "teamId", "ORtg", "DRtg", "NRtg", "poss"]
    df1 = df1[cols]
    df1["Win"] = df1["NRtg"] > 0
    df1["Loss"] = df1["NRtg"] < 0
    df2 = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df2 = df2.rename(
        columns={
            "GAME_ID": "gameId",
            "TEAM_ID": "teamId",
            "TEAM_ABBREVIATION": "teamTricode",
            "GAME_DATE": "gameDate",
            "TEAM_NAME": "teamName",
        }
    )
    cols2 = ["gameId", "teamId", "teamTricode", "gameDate"]
    df2 = df2[cols2]
    df2["gameId"] = df2["gameId"].astype(int)
    # df33 = pd.read_csv(box_DIR + f"NBA_BoxScores_4Factor_{season}.csv")
    # df33 = df33.rename(columns={"effectiveFieldGoalPercentage":"eFG1","freeThrowAttemptRate":"FTA1","teamTurnoverPercentage":"TOV1","offensiveReboundPercentage":"OREB1","oppEffectiveFieldGoalPercentage":"eFG2","oppFreeThrowAttemptRate":"FTA2","oppTeamTurnoverPercentage":"TOV2","oppOffensiveReboundPercentage":"OREB2"})
    # cols3 = ["gameId","teamId","eFG1","FTA1","TOV1","OREB1","eFG2","FTA2","TOV2","OREB2"]
    # df33 = df33[cols3]
    # df32 = pd.merge(df2,df1,on=["gameId","teamId"])
    # df3 = pd.merge(df32,df33,on=["gameId","teamId"])
    df3 = pd.merge(df2, df1, on=["gameId", "teamId"])
    df3 = df3.rename(columns={"gameDate": "Date"})
    df3["Date"] = pd.to_datetime(df3["Date"], format="%Y-%m-%d")
    df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
    df_teams = df_teams.rename(columns={"nameTeam": "Team"})
    df3 = pd.merge(df3, df_teams)
    df4 = df3.sort_values(by=["teamTricode", "Date"]).reset_index(drop=True)
    df6 = df3.sort_values(by=["Date"]).reset_index(drop=True)
    teams = df6["teamTricode"].unique()

    dfa = []
    dfa2 = []
    for team in teams:
        df5 = df4[df4["teamTricode"] == team].reset_index(drop=True)
        df5["tposs"] = df5["poss"].cumsum()
        # vals = ["ORtg","DRtg","eFG1","FTA1","TOV1","OREB1","eFG2","FTA2","TOV2","OREB2"]
        vals = ["ORtg", "DRtg"]
        for val in vals:
            df5 = rolling_val(df5, val)
        df5["rNRtg"] = df5["rORtg"] - df5["rDRtg"]
        df5["Wins"] = df5["Win"].cumsum()
        df5["Losses"] = df5["Loss"].cumsum()
        df5["Games_Played"] = df5["Wins"] + df5["Losses"]
        df5["Win_Frac"] = df5["Wins"] / df5["Games_Played"]
        df5["Rem_Wins"] = df5["Wins"].iloc[-1] - df5["Wins"]
        df5["Rem_Losses"] = df5["Losses"].iloc[-1] - df5["Losses"]
        df5["Rem_Games"] = df5["Games_Played"].iloc[-1] - df5["Games_Played"]
        df5["Rem_Win_Frac"] = df5["Rem_Wins"] / df5["Rem_Games"]
        rvals = ["r" + val for val in vals]
        for val in rvals[2:]:
            df5[val] = df5[val].round(3)
        df5["rORtg"] = df5["rORtg"].round(2)
        df5["rDRtg"] = df5["rDRtg"].round(2)
        df5["rNRtg"] = df5["rNRtg"].round(2)
        df5["Win_Frac"] = df5["Win_Frac"].round(4)
        df5["Rem_Win_Frac"] = df5["Rem_Win_Frac"].round(4)
        df5["Win_Percent"] = 100 * df5["Win_Frac"]
        df5["Rem_Win_Percent"] = 100 * df5["Rem_Win_Frac"]
        df8 = df5.tail(1)
        if remove_games_start > 0:
            df5 = df5.iloc[remove_games_start:]
        if remove_games_end > 0:
            df5 = df5.iloc[:-remove_games_end]
        df5 = df5.reset_index(drop=True)
        dfa.append(df5)
        dfa2.append(df8)

    data = pd.concat(dfa)
    dataf = pd.concat(dfa2)
    data["teamDup"] = data["teamTricode"]
    
    dates = df4["Date"].unique()
    dfb = []
    for date in dates:
        df7 = df6[df6["Date"] <= date].reset_index(drop=True)
        df7["tposs"] = df7["poss"].cumsum()
        vals = ["ORtg", "DRtg"]
        for val in vals:
            df7 = rolling_val(df7, val)
        df7["rNRtg"] = df7["rORtg"] - df7["rDRtg"]
        df7["rORtg"] = df7["rORtg"].round(2)
        df7["rDRtg"] = df7["rDRtg"].round(2)
        df7["rNRtg"] = df7["rNRtg"].round(2)
        dfb.append(df7.tail(1))
    data_avg = pd.concat(dfb).reset_index(drop=True)
    data_avg["teamTricode"] = "avg"
    data_avg["teamDup"] = data_avg["teamTricode"]
    data_avg = data_avg.drop(columns=["teamTricode"])

    return data, dataf, data_avg, df1, df2, df3

In [ ]:
season = "2023"
data, dataf, data_avg, df1, df2, df3 = get_rolling_ratings(season)
Date = "2023-11-07"
data = data[data["Date"] > Date]
data_avg = data_avg[data_avg["Date"] > Date]
data1 = data.copy().drop(columns=["teamTricode"])

In [ ]:
sdfsd

## OFF

In [ ]:
p = (
    ggplot(data)
    + geom_smooth(
        data=data1,
        mapping=aes(x="Date", y="rORtg", group="teamDup"),
        method="loess",
        color="lightgrey",
        se=False,
        size=0.5,
    )
    + geom_smooth(
        data=data_avg,
        mapping=aes(x="Date", y="rORtg", group="teamDup"),
        method="loess",
        color="red",
        se=False,
        size=0.5,
    )
    + geom_smooth(
        aes(x="Date", y="rORtg", group="teamTricode"),
        method="loess",
        color="Black",
        se=False,
        size=1,
    )
    + ylim(100, 125)
    + scale_x_date(date_labels="%b-%d", date_breaks="2 week")
    + labs(
        title="NBA Rolling Team Offensive Ratings as of " + datetime.today().strftime('%B %d, %Y'),
        subtitle="x-axis shows the day and y-axis shows Off Rtg till that day | Red line is league avg Off Rtg",
        caption="@SravanNBA | source:nba.com/stats",
    )
    + facet_wrap(facets="~ teamTricode")
    + theme_sra
)
p.save(fig_DIR + "Rolling ORtg.png", width=10, height=10, dpi=300)
p.draw()

## DEF

In [ ]:
p = (
    ggplot(data)
    + geom_smooth(
        data=data1,
        mapping=aes(x="Date", y="rDRtg", group="teamDup"),
        method="loess",
        color="lightgrey",
        se=False,
        size=0.5,
    )
    + geom_smooth(
        data=data_avg,
        mapping=aes(x="Date", y="rDRtg", group="teamDup"),
        method="loess",
        color="red",
        se=False,
        size=0.5,
    )
    + geom_smooth(
        aes(x="Date", y="rDRtg", group="teamTricode"),
        method="loess",
        color="Black",
        se=False,
        size=1,
    )
    + ylim(100, 125)
    + scale_x_date(date_labels="%b-%d", date_breaks="2 week")
    + labs(
        title="NBA Rolling Team Defensive Ratings as of" + datetime.today().strftime('%B %d, %Y'),
        subtitle="x-axis shows the day and y-axis shows Def Rtg till that day | Red line is league avg Def Rtg",
        caption="@SravanNBA | source:nba.com/stats",
    )
    + facet_wrap(facets="~ teamTricode")
    + theme_sra
)
p.save(fig_DIR + "Rolling DRtg.png", width=10, height=10, dpi=300)
p.draw()

## NET

In [ ]:
p = (
    ggplot(data)
    + geom_smooth(
        data=data1,
        mapping=aes(x="Date", y="rNRtg", group="teamDup"),
        color="lightgrey",
        se=False,
        size=0.5,
    )
    + geom_smooth(
        aes(x="Date", y="rNRtg", group="teamTricode"),
        color="Black",
        se=False,
        size=1,
    )
    + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
    # + ylim(-20, 20)
    # + scale_x_date(date_labels = "%b-%d",date_breaks = "2 week")
    # + ylim(-10, 10)
    + scale_x_date(date_labels="%b-%d", date_breaks="2 week")
    + labs(
        title="NBA Rolling Team Net Ratings as of " + datetime.today().strftime('%B %d, %Y'),
        subtitle="x-axis shows the day and y-axis shows Net Rating till that day",
        caption="@SravanNBA | source:nba.com/stats",
    )
    + facet_wrap(facets="~ teamTricode")
    + theme_sra
)
p.save(fig_DIR + "Rolling NRtg_2023.png", width=10, height=10, dpi=300)
p.draw()

## Individual Team

In [ ]:
theme_idv = themes.theme_xkcd(base_size=11)  # , base_family="Tahoma")
theme_idv += theme(
    # plot_background = element_rect(fill = 'ghostwhite', color = "ghostwhite"),
    plot_title=element_text(face="bold", size=16),
    # strip_text = element_text(face = 'bold',size=10),
    # plot_caption = element_text(size=10),
    # plot_subtitle = element_text(size=12),
    # axis_text_x = element_text(size=8),
    # axis_text_y = element_text(size=8),
    # axis_title_x = element_text(size=12),
    # axis_title_y = element_text(size=12)
)

In [ ]:
team = "BOS"
for team in data["teamTricode"].unique():
#     if team not in ["SAS","BOS"]:
    data2 = data[data["teamTricode"] == team].reset_index(drop=True)
    var = "rNRtg"
    title = data2["Team"][0] + " Rolling Net Rating 2023-24"
    subtitle = "x-axis shows the day and y-axis shows Off Rtg | League Avg is red"

    p = (
        ggplot(data2)
        + geom_smooth(
            data=data1,
            mapping=aes(x="Date", y=var, group="teamDup"),
            method="loess",
            color="lightgrey",
            se=False,
            size=0.5,
        )
        + geom_smooth(
            aes(x="Date", y=var, group="teamTricode"),
            # method="loess",
            color="Black",
            se=False,
        )
        + geom_point(
            aes(x="Date", y=var, group="teamTricode"),
            color="Black",
        )
        # + geom_smooth(data= data_avg,mapping=aes(x="Date", y=var,group = "teamDup"),method="loess",color="red", se=False, size=0.5)
        + geom_hline(yintercept=0, linetype="dashed", color="red", size=0.5)
        # + ylim(-10, 10)
        + scale_x_date(date_labels="%b-%d", date_breaks="1 week")
        # + ylim(100, 130)
        + labs(
            title=title,
            subtitle=subtitle,
            caption="@SravanNBA | source:nba.com/stats",
        )
        + theme_idv
    )
    # p.save(fig_DIR + "Rolling_" + var + "_" + team + ".png", width=6, height=5, dpi=300)
    p.draw()